## Human-in-the-Loop - Approval Workflows
### Pause Execution for Human Approval
### Protect User with PII Gaurdrail
The guardrail node detects and blocks the following PII types:
- **SSN**: Social Security Numbers (123-45-6789)
- **Credit Card**: Card numbers (1234-5678-9012-3456)
- **Mobile Number**: Phone numbers (+1-234-567-8900, (234) 567-8900)
- **Email**: Email addresses (user@example.com)
- **URL/Link**: Web links (http://example.com, www.example.com)

Learning Objectives:
- Use interrupt() to pause execution
- Get human approval before actions
- Resume with Command API
- Protect user identities with PII

#### Real-World Use Cases:
1. **Financial Systems**: Approve large transactions
2. **Content Moderation**: Review AI content before publishing
3. **Customer Support**: Review sensitive responses
4. **DevOps**: Approve deployments

In [ ]:
from typing_extensions import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.types import Command, interrupt
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
import os
import re

# Configuration
BASE_URL = "http://localhost:11434"
MODEL_NAME = "qwen3"

In [ ]:
# =============================================================================
# State Definition
# =============================================================================

class AgentState(TypedDict):
    """State for interrupt workflows."""
    messages: Annotated[list, operator.add]


In [ ]:
# =============================================================================
# Transfer Money Tool
# =============================================================================

@tool
def transfer_money(amount: int, recipient: str) -> str:
    """
    Transfer money. Large transfers require approval.

    Args:
        amount: Amount in dollars
        recipient: Recipient name
    """
    # Interrupt for large amounts
    if amount > 1000:
        # Pause and wait for human decision
        approval = interrupt({
            "type": "approval_required",
            "amount": amount,
            "recipient": recipient
        })

        # Check decision
        if approval.get("decision") != "approve":
            return "Transfer cancelled"

    # Execute transfer
    return f"Transferred ${amount} to {recipient}"


In [ ]:
# =============================================================================
# Guardrail Node - PII Detection
# =============================================================================

def guardrail_node(state: AgentState) -> dict:
    """Validate input to detect and block PII (Personally Identifiable Information)."""
    last_message = state["messages"][-1].content
    
    # PII Pattern Definitions
    patterns = {
        "SSN": r'\b\d{3}-\d{2}-\d{4}\b',  # SSN: 123-45-6789
        "Credit Card": r'\b\d{4}[\s-]?\d{4}[\s-]?\d{4}[\s-]?\d{4}\b',  # Credit Card: 1234-5678-9012-3456
        "Mobile Number": r'\b(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b',  # Mobile: +1-234-567-8900
        "Email": r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Email: user@example.com
        "URL/Link": r'https?://[^\s]+|www\.[^\s]+'  # URL: http://example.com or www.example.com
    }
    
    # Check for PII patterns
    for pii_type, pattern in patterns.items():
        if re.search(pattern, last_message):
            return {
                "messages": [SystemMessage(
                    content=f"Request blocked: Contains {pii_type}. Please don't share sensitive personal information."
                )]
            }
    
    # Input is safe, pass through
    return {"messages": []}


# =============================================================================
# Agent Node
# =============================================================================

def agent_node(state: AgentState) -> dict:
    """Agent that uses transfer_money tool."""
    llm = ChatOllama(model=MODEL_NAME, base_url=BASE_URL)
    tools = [transfer_money]
    llm_with_tools = llm.bind_tools(tools)

    messages = [
        SystemMessage(content="You are a financial assistant. Use transfer_money tool to transfer funds."),
        *state["messages"]
    ]

    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

In [ ]:
def should_continue(state: AgentState):
    """Route to tools or end."""
    last = state["messages"][-1]
    if hasattr(last, "tool_calls") and last.tool_calls:
        return "tools"
    return END


def guardrail_router(state: AgentState):
    """Route based on guardrail check."""
    last = state["messages"][-1]
    # If last message is SystemMessage (error), end workflow
    if isinstance(last, SystemMessage):
        return END
    # Otherwise continue to agent
    return "agent"

In [ ]:
# # Clean up old database
# if os.path.exists("chatbot.db"):
#     os.remove("chatbot.db")


# Use the checkpointer defined above
import sqlite3
os.makedirs('db', exist_ok=True)
db_path = 'db/checkpoints.db'
conn = sqlite3.connect(db_path, check_same_thread=False)
checkpointer = SqliteSaver(conn)

In [ ]:
# =============================================================================
# Graph Construction
# =============================================================================

def create_agent():
    """Create workflow with guardrails and interrupts."""
    builder = StateGraph(AgentState)

    # Add nodes
    builder.add_node("guardrail", guardrail_node)
    builder.add_node("agent", agent_node)
    builder.add_node("tools", ToolNode([transfer_money]))

    # Add edges
    builder.add_edge(START, "guardrail")
    builder.add_conditional_edges("guardrail", guardrail_router, ["agent", END])
    builder.add_conditional_edges("agent", should_continue, ["tools", END])
    builder.add_edge("tools", "agent")

    return builder.compile(checkpointer=checkpointer)

In [ ]:
agent = create_agent()
agent

In [ ]:
# =============================================================================
# Demo 1: Small Transfer (No Interrupt)
# =============================================================================

config = {"configurable": {"thread_id": "demo-1"}}

result = agent.invoke({
    "messages": [HumanMessage(content="Transfer 500 to Laxmi Kant")]
}, config)

result

In [ ]:
# =============================================================================
# Demo 2: Large Transfer - Approval Required
# =============================================================================

config = {"configurable": {"thread_id": "demo-2"}}

# Will interrupt
result = agent.invoke({
    "messages": [HumanMessage(content="Transfer 5000 to Laxmi Kant")]
}, config)

result

In [ ]:
# Check if interrupted
if "__interrupt__" in result:
    interrupt_data = result["__interrupt__"][0]

    # let's use approve for this demo
    # Resume with approval
    result = agent.invoke(
        Command(resume={"decision": "approve"}), # approved
        config
    )

    print(f"Result: {result['messages'][-1].content}")


In [ ]:
# =============================================================================
# Demo 3: Blocked by Guardrail - Email Detection
# =============================================================================

config = {"configurable": {"thread_id": "demo-3"}}

result = agent.invoke({
    "messages": [HumanMessage(content="Transfer 500 to udemy@kgptalkie.com")]
}, config)

print(f"Guardrail Result: {result['messages'][-1].content}")

In [ ]:
# =============================================================================
# Demo 4: Blocked by Guardrail - Credit Card Detection
# =============================================================================

config = {"configurable": {"thread_id": "demo-4"}}

result = agent.invoke({
    "messages": [HumanMessage(content="Transfer 500 using card 1234-5678-9012-3456")]
}, config)

print(f"Guardrail Result: {result['messages'][-1].content}")

In [ ]:
# =============================================================================
# Demo 4b: Blocked by Guardrail - Mobile Number Detection
# =============================================================================

config = {"configurable": {"thread_id": "demo-4b"}}

result = agent.invoke({
    "messages": [HumanMessage(content="Transfer 500 to +1-234-567-8900")]
}, config)

print(f"Guardrail Result: {result['messages'][-1].content}")

In [ ]:
# =============================================================================
# Demo 4c: Blocked by Guardrail - URL Detection
# =============================================================================

config = {"configurable": {"thread_id": "demo-4c"}}

result = agent.invoke({
    "messages": [HumanMessage(content="Transfer 500 and send receipt to https://example.com/receipt")]
}, config)

print(f"Guardrail Result: {result['messages'][-1].content}")

In [ ]:
# =============================================================================
# Demo 5: Interrupt - Rejection Flow
# =============================================================================

config = {"configurable": {"thread_id": "demo-5"}}

# Start large transfer
result = agent.invoke({
    "messages": [HumanMessage(content="Transfer 3000 to Laxmi Kant")]
}, config)

# Check if interrupted
if "__interrupt__" in result:
    print("Transfer requires approval:")
    print(f"  Amount: ${result['__interrupt__'][0].value['amount']}")
    print(f"  Recipient: {result['__interrupt__'][0].value['recipient']}")
    
    # Reject the transfer
    result = agent.invoke(
        Command(resume={"decision": "reject"}),
        config
    )
    
    print(f"\nFinal Result: {result['messages'][-1].content}")

## Key Features Demonstrated



### 2. Human-in-the-Loop Interrupts
- **Conditional Interrupts**: Only triggers for transactions > $1000
- **Approval Flow**: Pauses execution and waits for human decision
- **Resume Options**: Supports both approval and rejection paths

### 3. State Management
- **Thread Isolation**: Each demo uses separate thread IDs
- **Persistence**: Maintains conversation state across interrupts
- **Checkpoint Recovery**: Can resume from any interrupt point

### Workflow Flow:
```
START → Guardrail (PII Check) → Agent → Tools → Agent → END
         ↓ (PII detected)               ↓ (interrupt)
         END                             WAIT → Resume
```